In [24]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './spvae_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

Loaded data from: ./spvae_vectors_drags.csv | Columns = 131 / 131 | Rows = 439 -> 439


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128,drag
61,-0.996583,0.253377,2.345441,-0.989086,-2.179726,4.450816,-0.333054,-0.833110,0.170525,0.681121,...,1.321793,-0.977492,0.493258,-2.447803,-0.851386,0.003659,-0.704168,0.301570,0.179546,0.375
354,-0.107085,1.786517,1.586284,-1.288352,-1.469519,1.856284,0.022899,0.707441,0.195338,0.934119,...,0.785785,-1.589034,-3.525021,-4.223967,-2.653476,0.012422,0.597222,0.977202,-0.662360,0.374
358,-3.295154,-1.133777,-0.713116,0.946432,0.027521,0.023506,0.565126,-1.224105,-0.519147,0.146645,...,2.272549,-0.996835,5.970934,0.562599,3.063003,-0.003317,1.284646,-0.388674,0.367492,0.435
275,-0.809618,-1.043127,0.594460,1.935878,0.173827,1.941216,-0.689809,-1.535557,1.745006,0.349334,...,0.602183,-0.519529,0.047703,1.788710,-0.448435,-0.003273,-0.998566,-0.302207,0.684875,0.437
18,-0.572277,-0.328782,0.444743,0.411545,-1.628947,3.242414,-0.008619,-0.790840,0.288297,0.432827,...,2.103731,-2.167353,-1.278066,-1.638192,2.981116,0.002460,-0.616031,-0.505367,0.432376,0.367


In [25]:
train_drags = train_df["drag"]
train_index = train_df["i"]
for index, drag in zip(train_index, train_drags):
    print(index, "," ,drag)

61 , 0.375
354 , 0.374
358 , 0.435
275 , 0.437
18 , 0.367
107 , 0.36
57 , 0.456
430 , 0.386
374 , 0.345
179 , 0.345
287 , 0.341
51 , 0.55
395 , 0.48
133 , 0.371
233 , 0.427
210 , 0.478
151 , 0.294
375 , 0.411
24 , 0.538
311 , 0.356
152 , 0.429
369 , 0.478
8 , 0.317
90 , 0.439
47 , 0.418
305 , 0.423
421 , 0.377
253 , 0.418
167 , 0.322
315 , 0.355
345 , 0.489
366 , 0.42
41 , 0.51
359 , 0.453
269 , 0.362
56 , 0.375
193 , 0.408
283 , 0.513
204 , 0.315
224 , 0.333
316 , 0.571
19 , 0.375
150 , 0.33
118 , 0.298
58 , 0.341
223 , 0.359
1 , 0.393
226 , 0.403
124 , 0.339
232 , 0.404
278 , 0.404
342 , 0.344
26 , 0.4
390 , 0.326
379 , 0.417
81 , 0.384
236 , 0.432
100 , 0.374
383 , 0.315
148 , 0.334
3 , 0.337
371 , 0.329
123 , 0.446
231 , 0.348
248 , 0.37
39 , 0.379
120 , 0.439
273 , 0.317
29 , 0.436
378 , 0.371
352 , 0.429
154 , 0.362
407 , 0.347
131 , 0.357
261 , 0.387
262 , 0.402
382 , 0.468
10 , 0.425
88 , 0.402
372 , 0.445
73 , 0.417
234 , 0.516
106 , 0.307
213 , 0.334
364 , 0.356
206 , 0.434
3

In [26]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [27]:
import os
save_path = './agModels-spvae'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-spvae/learner.pkl
Saving ./agModels-spvae/predictor.pkl
Beginning

In [28]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_119,dim_120,dim_121,dim_122,dim_123,dim_124,dim_125,dim_126,dim_127,dim_128
46,-1.675357,0.774387,-2.380979,-0.728630,-1.412328,1.705713,-0.913276,-0.664502,-0.036643,0.697300,...,2.785418,-0.100383,-1.418045,-5.795298,-0.239288,-3.270577,0.005218,2.296032,-0.461153,1.085652
101,-0.058659,0.094974,-0.148768,-1.709858,0.231730,0.003423,-0.203162,0.288094,-0.420197,-2.209749,...,-2.907042,0.220606,0.879496,-1.250059,-0.911256,2.207671,-0.007707,1.828314,-0.745860,-1.381517
175,0.408514,-0.318794,-0.195711,-1.258788,-0.074261,-3.524118,0.243094,-0.282567,0.518098,1.104262,...,-2.164850,-2.873618,0.817046,1.877096,-1.402977,-0.992778,-0.002622,1.483371,0.380046,0.484612
9,-2.754269,-1.133549,-1.814455,0.036195,-0.159687,2.911953,-0.640023,-0.894014,0.440514,0.164435,...,-2.664808,1.762599,-0.071357,3.233031,-0.915575,2.781991,-0.008800,0.508250,-0.724321,-0.320688
136,-1.054854,0.100548,0.473871,3.140375,-0.449581,2.196428,0.303511,-0.011313,0.375206,-0.866014,...,3.855908,0.273155,-2.094910,-3.537341,1.143077,-2.242801,-0.000889,0.217993,-0.597952,1.817741


In [29]:
test_drags = test_df["drag"]
test_index = test_df["i"]
for index, drag in zip(test_index, test_drags):
    print(index, "," ,drag)

46 , 0.548
101 , 0.37
175 , 0.38
9 , 0.431
136 , 0.38
381 , 0.486
239 , 0.435
63 , 0.496
401 , 0.414
238 , 0.314
168 , 0.326
409 , 0.39
97 , 0.361
368 , 0.541
215 , 0.429
200 , 0.483
172 , 0.426
169 , 0.409
282 , 0.369
134 , 0.32
243 , 0.398
145 , 0.331
166 , 0.327
286 , 0.373
6 , 0.431
165 , 0.371
75 , 0.435
146 , 0.336
25 , 0.388
184 , 0.419
12 , 0.445
161 , 0.35
89 , 0.364
362 , 0.413
72 , 0.413
77 , 0.41
147 , 0.32
222 , 0.389
412 , 0.401
110 , 0.347
290 , 0.631
21 , 0.375
178 , 0.436
420 , 0.395
257 , 0.406
304 , 0.357
52 , 0.407
48 , 0.435
177 , 0.363
252 , 0.403
436 , 0.375
260 , 0.374
406 , 0.464
153 , 0.41
281 , 0.394
426 , 0.35
400 , 0.456
174 , 0.395
144 , 0.345
170 , 0.325
192 , 0.339
349 , 0.424
36 , 0.361
242 , 0.414
185 , 0.395
398 , 0.364
241 , 0.309
277 , 0.336
86 , 0.365
408 , 0.446
351 , 0.438
339 , 0.374
417 , 0.446
416 , 0.453
54 , 0.417
188 , 0.38
79 , 0.39
255 , 0.341
411 , 0.355
328 , 0.454
300 , 0.376
196 , 0.428
7 , 0.307
173 , 0.398
5 , 0.372
55 , 0.44
428 , 

In [30]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_spvae.log', 'w') as f:
    f.write(log_output.stdout)

Loading: ./agModels-spvae/predictor.pkl
Loading: ./agModels-spvae/learner.pkl
Loading: ./agModels-spvae/models/trainer.pkl
Loading: ./agModels-spvae/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-spvae/models/NeuralNetTorch_BAG_L2/model.pkl
Loading: ./agModels-spvae/models/WeightedEnsemble_L3/m

In [35]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 128 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [34]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

Loading: ./agModels-spvae/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/WeightedEnsemble_L2/model.pkl


0.39116281270980835
0.38422268629074097
0.4321916103363037
0.4221478998661041
0.381976842880249
0.37929338216781616
0.44311171770095825
0.3903323709964752
0.38242995738983154
0.3697527348995209
0.3696485161781311
0.48752298951148987
0.44381165504455566
0.38811272382736206
0.4178387224674225
0.4455588459968567
0.33955118060112
0.4062100350856781
0.4758799076080322
0.3802383542060852
0.41610413789749146
0.44242262840270996
0.35412436723709106
0.4298839569091797
0.4077208340167999
0.41002267599105835
0.38871702551841736
0.411051869392395
0.3525809645652771
0.36932802200317383
0.446757435798645
0.4205259084701538
0.4632236361503601
0.42609119415283203
0.3690805435180664
0.3842049241065979
0.41131722927093506
0.45698171854019165
0.353503555059433
0.3623332977294922
0.4831129312515259
0.38434967398643494
0.3578150272369385
0.3488399088382721
0.36573535203933716
0.3847488760948181
0.3968857526779175
0.4063284397125244
0.36181819438934326
0.39586013555526733
0.3985598683357239
0.37376213073730

In [36]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

Loading: ./agModels-spvae/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-spvae/models/WeightedEnsemble_L2/model.pkl


0.3715846538543701
0.3892876207828522
0.3662589490413666
0.4449158012866974
0.4008152484893799
0.4123873710632324
0.4056609272956848
0.40893423557281494
0.40943390130996704
0.39007431268692017
0.41601961851119995
0.40324831008911133
0.3807868957519531
0.4320763945579529
0.39484596252441406
0.3994584083557129
0.37342697381973267
0.41242337226867676
0.4068812429904938
0.36535704135894775
0.4233607351779938
0.4209880232810974
0.3790907859802246
0.3989671766757965
0.4112382233142853
0.39417600631713867
0.41431230306625366
0.389351487159729
0.3853309750556946
0.3866821527481079
0.4274556636810303
0.3762912154197693
0.3949410319328308
0.41964006423950195
0.403073251247406
0.3803277909755707
0.3869140148162842
0.40557241439819336
0.3959423005580902
0.39824503660202026
0.4232049286365509
0.3929121494293213
0.42545002698898315
0.43621018528938293
0.4298994839191437
0.4001341164112091
0.37912899255752563
0.3720305263996124
0.44284701347351074
0.4230244755744934
0.3976234197616577
0.3722046315670